In [1]:
"""import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Input, Dense, Conv2DTranspose, UpSampling2D, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained U-Net model
base_model = VGG19(weights='imagenet', include_top=False, input_tensor=Input(shape=(224,224,3)))

# Freeze the base model
base_model.trainable = False

# Extract features from pre-trained model
encoder_outputs = [base_model.get_layer(name).output for name in ["block1_pool", "block2_pool", "block3_pool", "block4_pool", "block5_pool"]]

# Create the decoder part of the U-Net model
x = encoder_outputs[-1]
for i in range(4, 0, -1):
    x = Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same', activation='relu')(x)
    x = Concatenate()([x, encoder_outputs[i-1]])
    x = Conv2DTranspose(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(256, (3, 3), activation='relu', padding='same')(x)

x = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)

# Add a GlobalAveragePooling2D layer to reduce the spatial dimensions of the feature map
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with a sigmoid activation to predict the cancer class
output_layer = Dense(1, activation='sigmoid')(x)

# Create the final model by combining the encoder and decoder
model = Model(inputs=base_model.input, outputs=output_layer)
"""

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Create the AlexNet model
model = Sequential()

# Layer 1: Convolutional layer with 96 filters of size 11x11 and stride of 4
model.add(Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))

# Layer 2: Max pooling layer with size 3x3 and stride of 2
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# Layer 3: Convolutional layer with 256 filters of size 5x5 and padding
model.add(Conv2D(256, (5, 5), padding='same', activation='relu'))

# Layer 4: Max pooling layer with size 3x3 and stride of 2
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# Layer 5: Convolutional layer with 384 filters of size 3x3 and padding
model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

# Layer 6: Convolutional layer with 384 filters of size 3x3 and padding
model.add(Conv2D(384, (3, 3), padding='same', activation='relu'))

# Layer 7: Max pooling layer with size 3x3 and stride of 2
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# Layer 8: Flatten layer to convert the output of the previous layer to a 1D array
model.add(Flatten())

# Layer 9: Fully connected layer with 512 neurons
model.add(Dense(512, activation='relu'))

# Layer 10: Dropout layer to reduce overfitting
model.add(Dropout(0.5))

# Layer 11: Fully connected layer with 512 neurons
model.add(Dense(512, activation='relu'))

# Layer 12: Dropout layer to reduce overfitting
model.add(Dropout(0.5))

# Layer 13: Output layer with 1 neuron for binary classification
model.add(Dense(1, activation='sigmoid'))

# Print the model summary
model.summary()


In [2]:
cancerFolder = 'archive/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor'
nonCancerFolder = 'archive/Brain Tumor Data Set/Brain Tumor Data Set/Healthy'

In [3]:
import glob
import numpy as np
from sklearn.utils import shuffle

cancerFiles = glob.glob('{}*/*.jpg'.format(cancerFolder), recursive=True)
nonCancerFiles = glob.glob('{}*/*.jpg'.format(nonCancerFolder), recursive=True)
imageSamplesFiles = cancerFiles + nonCancerFiles
y = np.concatenate((np.ones(len(cancerFiles)), np.zeros(len(nonCancerFiles))))
imageSamplesFiles, y = shuffle(imageSamplesFiles, y)

In [4]:
from sklearn.model_selection import train_test_split as trainTestSplit

xTrain, xTest, yTrain, yTest = trainTestSplit(imageSamplesFiles, y, test_size=0.2, random_state=42)
#xTrain, xVal, yTrain, yVal = trainTestSplit(xTrain, yTrain, test_size=0.2, random_state=42)

In [5]:
import cv2
import numpy as np

def resize_images(imgs):
    resized_images = []
    for img_path in imgs:
        # Read the image using cv2.imread()
        img = cv2.imread(img_path)
        # Resize the image using cv2.resize()
        resized_img = cv2.resize(img, (224, 224))
        # Append the resized image to the list
        resized_images.append(resized_img)
    return np.array(resized_images)

In [6]:
xTrain=resize_images(xTrain)
xTest=resize_images(xTest)

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
from keras.callbacks import ModelCheckpoint

# define the filepath for the best model to be saved
filepath="alexnet_model.hdf5"

# define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [9]:
# train the model using the callback
history = model.fit(xTrain, yTrain, epochs=100, batch_size=500, validation_data=(xTest, yTest), callbacks=[checkpoint])

Epoch 1/2
113/113 [==============================] - ETA: 0s - loss: 133.8574 - accuracy: 0.5139  
Epoch 1: val_loss improved from inf to 0.68876, saving model to best_model.hdf5
113/113 [==============================] - 24000s 213s/step - loss: 133.8574 - accuracy: 0.5139 - val_loss: 0.6888 - val_accuracy: 0.5443
Epoch 2/2
113/113 [==============================] - ETA: 0s - loss: 0.6871 - accuracy: 0.5391  
Epoch 2: val_loss improved from 0.68876 to 0.68676, saving model to best_model.hdf5
113/113 [==============================] - 26001s 230s/step - loss: 0.6871 - accuracy: 0.5391 - val_loss: 0.6868 - val_accuracy: 0.5443


In [18]:
"""#Fine_tuning
base_model.trainable = True
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.00001),# Very low learning rate
              loss='binary_crossentropy',
              metrics=['accuracy'])
"""

In [19]:
"""# define the filepath for the best model to be saved
filepath="alexnet_ft_model.hdf5"

# define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')"""

In [20]:
"""# train the model using the callback
history_f = model.fit(xTrain, yTrain, epochs=50, batch_size=500, validation_data=(xTest, yTest), callbacks=[checkpoint])"""

113/113 [==============================] - ETA: 0s - loss: 0.6858 - accuracy: 0.5369  
Epoch 1: val_loss improved from inf to 0.68045, saving model to best_fine_tuned_model.hdf5
113/113 [==============================] - 39462s 350s/step - loss: 0.6858 - accuracy: 0.5369 - val_loss: 0.6804 - val_accuracy: 0.5443
